## Optimization for Machine Learning Project

This project revisits the course's notebooks by replacing the use of gradients and derivatives with **zeroth-order approximations**. These techniques are randomized and focus on obtaining good approximations in a probabilistic sense, bearing resemblance to finite-difference techniques.

### An MNIST-based regression problem

#### **Question 0**

Import the MNIST dataset and build your own subset by making the following changes:

1.  Select two digit classes $c_{1}$ and $c_{2}$ such that $\{c_{1}, c_{2}\} \neq \{0, 1\}$.
2.  Make sure the labels correspond to $0$ for one class and $1$ for the other class.
3.  Consider the images as vectors.

Given a subset of two classes in MNIST $\{(\bm{x_{i}}, y_{i})\}_{i=1}^{n}$ with $\bm{x_{i}} \in \mathbb{R}^{d}$, $d=28^{2}=784$ and $y_{i} \in \{0, +1\}$, we consider the classification problem:

$$\underset{\bm{w} \in \mathbb{R}^{d}}{\text{minimize}} \ f(\bm{w}) := \frac{1}{n} \sum_{i=1}^{n} f_{i}(\bm{w}), \quad f_{i}(\bm{w}) := \left(y_{i} - \frac{1}{1 + \exp(-\bm{x_{i}}^{T}\bm{w})}\right)^{2} \quad (1) \text{}$$

This problem is generally nonconvex. The function $t \mapsto \frac{1}{1 + \exp(-t)}$ is called the sigmoid function, and acts as an approximation to the sign function.

For any $i=1,...,n$, the gradient of $f_{i}$ is explicitly given by:
$$\nabla f_{i}(\bm{w}) = -\frac{2 \exp(\bm{x_{i}}^{T}\bm{w}) (\exp(\bm{x_{i}}^{T}\bm{w})(y_{i}-1) + y_{i})}{(1 + \exp(\bm{x_{i}}^{T}\bm{w}))^{3}} \bm{x_{i}} \quad (2) \text{}$$

In [2]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

In [3]:
# Download MNIST dataset

to_download = True

if not os.path.exists('./data'):
    os.makedirs('./data')
    to_download = True

transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, download=to_download, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=to_download, transform=transform) 
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


c1, c2 = 2, 3
labels_to_keep = [c1, c2]

train_mask = torch.tensor(
        [(target in labels_to_keep) for target in train_dataset.targets]
    )
test_mask = torch.tensor(
        [(target in labels_to_keep) for target in test_dataset.targets]
    )

train_dataset.data = train_dataset.data[train_mask]
train_dataset.data = train_dataset.data.view(-1, 28*28)
train_dataset.targets = train_dataset.targets[train_mask]
train_dataset.targets = torch.where(train_dataset.targets == c1, torch.tensor(0), torch.tensor(1))

test_dataset.data = test_dataset.data[test_mask]
test_dataset.data = test_dataset.data.view(-1, 28*28)
test_dataset.targets = test_dataset.targets[test_mask]
test_dataset.targets = torch.where(test_dataset.targets == c1, torch.tensor(0), torch.tensor(1))

n_train = train_dataset.data.shape[0]
n_test = test_dataset.data.shape[0]
d = 28 * 28

#### **Question 1** 

Given a data point $(\bm{x_{i}}, y_{i})$ from your dataset, use the Autograd framework described in the second lab session to implement a code for the function $f_{i}$ that enables computation of $\nabla f_{i}(\bm{x})$ for any $\bm{x}$ through automatic differentiation. Validate your implementation using the explicit formula (2).

In [4]:
def sigmoid(t):
    return 1 / (1 + torch.exp(-t))

In [5]:
def f_i(w, x_i, y_i):
    return (y_i - sigmoid(torch.dot(w, x_i)))**2

In [6]:
def grad_f_i(w, x_i, y_i):
    numerator = -2 * torch.exp(torch.dot(w, x_i)) * (torch.exp(torch.dot(w, x_i)) * (y_i - 1) + y_i)
    denominator = sigmoid(-torch.dot(w, x_i))**3
    return numerator / denominator * x_i

In [19]:
i = 1
x_i, y_i = train_dataset.data[i].float(), train_dataset.targets[i].float()
w = torch.zeros(d, requires_grad=True)

f_i_value = f_i(w, x_i, y_i)

f_i_value.backward()
grad_f_i_AD = w.grad
grad_f_i_explicit = grad_f_i(w.detach(), x_i, y_i)

In [20]:
print(w.grad)

tensor([ -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,
         -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.0000,  -0.00

In [22]:
print(grad_f_i_explicit)

tensor([   -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,    -0.,
           -0.,    -0.,    -0.,    -0., 

In [21]:
# Compare the results
tolerance = 1e-6
comparison_result = torch.allclose(grad_f_i_AD, grad_f_i_explicit, atol=tolerance)

print(f"Gradient via Autograd (g_auto):\n{grad_f_i_AD[:10]}") # Print first 10 elements for brevity
print(f"Gradient via Explicit Formula (g_explicit):\n{grad_f_i_explicit[:10]}")
print(f"Are the gradients numerically close? {comparison_result}")

Gradient via Autograd (g_auto):
tensor([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.])
Gradient via Explicit Formula (g_explicit):
tensor([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.])
Are the gradients numerically close? False


In [ ]:
# Compare the results
tolerance = 1e-6
comparison_result = torch.allclose(g_auto, g_explicit, atol=tolerance)

print(f"Gradient via Autograd (g_auto):\n{g_auto[:5]}") # Print first 5 elements for brevity
print(f"Gradient via Explicit Formula (g_explicit):\n{g_explicit[:5]}")
print(f"Are the gradients numerically close? {comparison_result}")

**Regularized version** We will also be interested in a regularized version of problem (1), of the form:

$$\underset{\bm{w} \in \mathbb{R}^{d}}{\text{minimize}} \ f(\bm{w}) + \lambda ||\bm{w}||_{1} \quad (3) \text{}$$
where $\lambda > 0$ and $||\bm{w}||_{1} = \sum_{j=1}^{d} |w_{j}|$.

### First-order Algorithms

#### **Question 2**

Adapt the code of gradient descent provided during the lab sessions (or use your own implementation) to run it on problem (1).

1.  What convergence rate is expected for gradient descent on this problem? Do you observe this rate empirically?
2.  Can you find a good constant value for the stepsize?

#### **Question 3**

Adapt the code of batch stochastic gradient provided during the lab sessions (or use your own implementation) to compare gradient descent and stochastic gradient on problem (1).

1.  Are your results consistent with what the theory predicts?
2.  Can you find a good constant stepsize choice for stochastic gradient?
3.  What appears to be the best value for the batch size on this problem?

#### **Question 4**

Adapt the code of Adagrad provided during the lab sessions (or use your own implementation) to include that method in the comparison. How does this method compare to the best stochastic gradient method from Question 3 on problem (1)?

#### **Question 5**

Consider the best method from Question 4, and apply it to problem (3) using proximal gradient (consider the implementation used as illustration in class, or implement your own). Can you find a value for $\lambda$ that yields a good yet sparse solution vector?

### Zeroth-order Algorithms

In this final section, we consider optimization techniques that do not rely on exact gradient calculations (let alone differentiation). Instead, one computes steps by querying only function values.

The classical iteration of such a method has the form:

$$w_{k+1} = w_{k} - \alpha_{k} \frac{f(w_{k} + h u_{k}) - f(w_{k})}{h} u_{k} \quad (4) \text{}$$
where $\alpha_{k} > 0$ is a stepsize, $h > 0$ plays the role of a finite-difference parameter and $u_{k} \sim \mathcal{N}(0, I_{d})$ is a Gaussian random vector. This approach was popularized by Nesterov and algorithms of the form (4) are nowadays referred to as **zeroth-order optimization methods**.

#### **Question 6**

Implement an algorithm based on iteration (4) and validate your implementation on the linear regression problem from the notebooks.

1.  Try different values for $h$ in order to find one that leads to convergence.
2.  Try different choices for $\alpha_{k}$ to obtain the best possible performance.

#### **Question 7**

It is worth comparing the proposed method (4) with both exact gradient descent (using exact derivatives) and a **finite-difference version** of the method:

$$w_{k+1} = w_{k} - \alpha_{k} g_{k} \quad (79)$$
where
$$g_{k} = \left[\frac{f(w_{k} + h e_{j}) - f(w_{k})}{h}\right]_{j=1,...,d} \quad (5) \text{}$$

Compare the performance of your algorithm (4) with that of gradient descent and the finite-difference method (5) on problem (1).

1.  How do the variants behave with **constant stepsize**? **Decreasing stepsizes**?
2.  Propose a **unit of comparison** (other than the number of iterations) for all three algorithms. Plot the behavior of the methods for a fixed budget of the cost you propose.

#### **Question 8**

In a stochastic setting, there are two ways to implement iteration (4).

**Variant 1 (Batch $f_{\mathcal{S}_{k}}$):** The first considers that a batch of indices $\mathcal{S}_{k}$ is given at every iteration, and performs:

$$w_{k+1} = w_{k} - \alpha_{k} \frac{f_{\mathcal{S}_{k}}(w_{k} + h u_{k}) - f_{\mathcal{S}_{k}}(w_{k})}{h} u_{k} \quad (6) \text{}$$
where $\forall \mathcal{S} \subset \{1,...,n\}$, $f_{\mathcal{S}}(\bm{w}) = \frac{1}{|\mathcal{S}|} \sum_{i \in \mathcal{S}} f_{i}(\bm{w})$.

**Variant 2 (Two Batches $f_{\mathcal{S}_{k}^{+}}, f_{\mathcal{S}_{k}}$):** The second approach considers that the same sample cannot be queried twice in a row, yielding the iteration:

$$w_{k+1} = w_{k} - \alpha_{k} \frac{f_{\mathcal{S}_{k}^{+}}(w_{k} + h u_{k}) - f_{\mathcal{S}_{k}}(w_{k})}{h} u_{k} \quad (7) \text{}$$
where $\mathcal{S}_{k}$ and $\mathcal{S}_{k}^{+}$ are random batches of same size.

Adapt the code from the previous questions to allow for iterations (6) and (7).

1.  Compare the variants with **batch size 1** with the deterministic method (4). Do you recover observations from the first-order part of this project?
2.  Assuming samples are drawn uniformly at random, find a good value for the **batch size** in both (6) and (7).

#### **Question 9**

1.  Propose an adaptation of your best stochastic zeroth-order variant to the **proximal setting**.
2.  Compare the resulting method with the algorithm from Question 5 (Proximal Gradient) on problem (3), with **exact gradients** and **finite-difference gradients**.

### References

[1] Y. Le Cun, L. Bottou, Y. Bengio, and P. Haffner. Gradient-based learning applied to document recognition. Proc. of the IEEE, 86(11):2278-2324, 1998.
[2] Yu. Nesterov. Random gradient-free minimization of convex functions. Technical Report 2011/1, CORE, Université Catholique de Louvain, 2011.
[3] Yu. Nesterov and V. Spokoiny. Random gradient-free minimization of convex functions. Found. Comput. Math., 17:527-566, 2017.